In [1]:
# import

%matplotlib inline

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
#import geopy.distance as distance


import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import json


In [2]:
## RAW DATA - no need to read

# load wapo
wapo_df = pd.read_csv('./data/fatal-police-shootings-data.csv')

# load MPV data
mpv_df = pd.read_excel(pd.ExcelFile('data/MPVDatasetDownload_CleanColumnNames.xlsx'), sheet_name='2013-2020 Police Killings')

# load county-wise demographic info from US census
county_df = pd.read_csv('data/acs2015_county_data.csv')

# load election data from HW 1
election_county_df = pd.read_csv('data/election2016_by_county.csv')

# lemas geocode
lemas_geocode = pd.read_csv('data/lemas_geocode.csv')

# load LEMAS
lemas_raw = pd.read_csv('data/LEMAS_2016.tsv', sep = '\t', index_col = 'LEAR_ID', header = 0, encoding = 'unicode_escape')

# rename some variables for merging - eg. merge county demographic info into LEMAS using FIPS
county_df['FIPS'] = county_df.CensusId
election_county_df['FIPS'] = election_county_df.fipscode


In [3]:
# read in clean victim list with matched ORI9 codes for each 

vic_df = pd.read_csv('data/victim_data_with_clean_ori9code.csv')

# load LEMAS
lemas_raw = pd.read_csv('data/LEMAS_2016.tsv', sep = '\t', index_col = 'LEAR_ID', header = 0, encoding = 'unicode_escape')
display(vic_df.head())
display(lemas_raw.shape)

,Unnamed: 0,id,name,date,armed_wapo,age,gender,race,city_wapo,state,...,race_N,race_O,race_W,geo_Rural,geo_Suburban,geo_Undetermined,geo_Urban,threat_attack,threat_other,threat_undetermined
0,0,3,Tim Elliot,2015-01-02,gun,53.0,M,A,Shelton,WA,...,0,0,0,1,0,0,0,1,0,0
1,1,4,Lewis Lee Lembke,2015-01-02,gun,47.0,M,W,Aloha,OR,...,0,0,1,0,1,0,0,1,0,0
2,2,5,John Paul Quintero,2015-01-03,unarmed,23.0,M,H,Wichita,KS,...,0,0,0,0,0,0,1,0,1,0
3,3,8,Matthew Hoffman,2015-01-04,toy weapon,32.0,M,W,San Francisco,CA,...,0,0,1,0,0,0,1,1,0,0
4,4,9,Michael Rodriguez,2015-01-04,nail gun,39.0,M,H,Evans,CO,...,0,0,0,0,1,0,0,1,0,0


(2784, 433)

In [4]:
# take a subset of lemas data:

# extract relevant stats for LEMAS - MORE SHOULD BE DONE HERE
def clean_lemas(lemas_raw):
    race_fields = ['WHITE','BLACK','HISP','AMIND','ASIAN','HAWPI','MULTI','UNK']
    gender_fields = ['MALE','FEM']

    # remove rows that do not have demographic info
    #lemas_raw = lemas_raw[(lemas_raw.PERS_MALE>=0) & (lemas_raw.PERS_FEMALE>=0) & (lemas_raw.PERS_WHITE_FEM>=0) & (lemas_raw.PERS_WHITE_MALE>=0)][:]


    lemas_raw['PERS_TOTAL'] = lemas_raw.PERS_MALE + lemas_raw.PERS_FEMALE

    # for now, take a fraction of the columns or it is overwhelming...
    # EDIT:  take agencyname, ORI9 (for later merging), and policy related columns
    lemas_new = lemas_raw[['AGENCYNAME','ORI9','POL_BWC','PERS_TRN_ACAD','PERS_TRN_FIELD','POL_COMP_EXTINV',
                          'POL_INV_INJRY','POL_INV_DTH','POL_INV_DCHG_GUN','PERS_CULTURE','PERS_CONFLICT']].copy()
    
    # categorical: 'POL_BWC', 'POL_COMP_EXTINV','POL_INV_INJRY','POL_INV_DTH',
    # 'POL_INV_DCHG_GUN','PERS_CULTURE','PERS_CONFLICT' (should all be int type)
    # yes is 1, no is 2 (need to be replaced by 0)
    cols = ['POL_BWC', 'POL_COMP_EXTINV','POL_INV_INJRY','POL_INV_DTH',\
            'POL_INV_DCHG_GUN','PERS_CULTURE','PERS_CONFLICT']
    lemas_new[cols] = lemas_new[cols].replace(2,0)
    
    # impute missing value marker -9 and -8 with 0 because the policy in question likely does NOT exist/get implemented at these agencies
    lemas_new=lemas_new.replace([-9,-8],0)
    
    lemas_new['PERS_TOTAL'] = lemas_raw.PERS_MALE + lemas_raw.PERS_FEMALE
    
    for this_race in race_fields:
        lemas_new['PERS_' + this_race] = lemas_raw['PERS_' + this_race + '_MALE'] + lemas_raw['PERS_' + this_race + '_FEM']
        lemas_new['FRAC_' + this_race] = lemas_new['PERS_' + this_race]/lemas_new.PERS_TOTAL
    

    # make LEMAS FIPS CODE NUMERIC for merging with demographic data 
#     lemas_new.FIPS = pd.to_numeric(lemas_new.FIPS,errors='coerce')
#     lemas_new = lemas_new.dropna()
    
    
    return lemas_new


lemas_df = clean_lemas(lemas_raw)

lemas_df.head()

,AGENCYNAME,ORI9,POL_BWC,PERS_TRN_ACAD,PERS_TRN_FIELD,POL_COMP_EXTINV,POL_INV_INJRY,POL_INV_DTH,POL_INV_DCHG_GUN,PERS_CULTURE,...,PERS_AMIND,FRAC_AMIND,PERS_ASIAN,FRAC_ASIAN,PERS_HAWPI,FRAC_HAWPI,PERS_MULTI,FRAC_MULTI,PERS_UNK,FRAC_UNK
LEAR_ID,,,,,,,,,,,,,,,,,,,,,
635592,DAVIS POLICE DEPARTMENT,CA0570100,1,888,900,0,0,1,0,0,...,0,0.0,6,0.1,0,0.0,0,0.000000,0,0.0
645110,WEST NEW YORK POLICE,NJ0091200,0,805,400,0,0,0,0,1,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0
631270,WESTOVER POLICE DEPARTMENT,WV0310400,1,640,640,0,1,1,1,1,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0
631316,BARABOO POLICE DEPARTMENT,WI0570200,1,720,640,1,0,1,1,0,...,0,0.0,0,0.0,0,0.0,1,0.034483,0,0.0
631684,OREGON POLICE DEPARTMENT,WI0137400,1,720,490,0,1,1,1,0,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0


In [6]:
# merge lemas 

vic_df_lemas = vic_df.merge(lemas_df,how='left',on='ORI9',suffixes=['_wapo','_lemas'])

vic_df_lemas.head()

,Unnamed: 0,id,name,date,armed_wapo,age,gender,race,city_wapo,state,...,PERS_AMIND,FRAC_AMIND,PERS_ASIAN,FRAC_ASIAN,PERS_HAWPI,FRAC_HAWPI,PERS_MULTI,FRAC_MULTI,PERS_UNK,FRAC_UNK
0,0,3,Tim Elliot,2015-01-02,gun,53.0,M,A,Shelton,WA,...,3,0.055556,0,0.000000,0,0.000000,0,0.0,0,0.000000
1,1,4,Lewis Lee Lembke,2015-01-02,gun,47.0,M,W,Aloha,OR,...,3,0.007463,16,0.039801,0,0.000000,0,0.0,3,0.007463
2,2,5,John Paul Quintero,2015-01-03,unarmed,23.0,M,H,Wichita,KS,...,2,0.002950,21,0.030973,0,0.000000,0,0.0,0,0.000000
3,3,8,Matthew Hoffman,2015-01-04,toy weapon,32.0,M,W,San Francisco,CA,...,7,0.002971,385,0.163413,140,0.059423,0,0.0,38,0.016129
4,4,9,Michael Rodriguez,2015-01-04,nail gun,39.0,M,H,Evans,CO,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000


In [7]:
vic_df_lemas.columns

Index(['Unnamed: 0', 'id', 'name', 'date', 'armed_wapo', 'age', 'gender',
       'race', 'city_wapo', 'state', 'signs_of_mental_illness', 'threat_level',
       'flee', 'body_camera', 'longitude', 'latitude', 'is_geocoding_exact',
       'wapo_id', 'street_address', 'city_mpv', 'state_code', 'zipcode',
       'county', 'police_department', 'ORI_agency_id', 'ORI9',
       'ORI9_match_type', 'is_male', 'is_tasered_and_shot', 'is_armed_gun',
       'is_armed_knife', 'is_unarmed', 'is_armed_other', 'race_A', 'race_B',
       'race_H', 'race_N', 'race_O', 'race_W', 'geo_Rural', 'geo_Suburban',
       'geo_Undetermined', 'geo_Urban', 'threat_attack', 'threat_other',
       'threat_undetermined', 'AGENCYNAME', 'POL_BWC', 'PERS_TRN_ACAD',
       'PERS_TRN_FIELD', 'POL_COMP_EXTINV', 'POL_INV_INJRY', 'POL_INV_DTH',
       'POL_INV_DCHG_GUN', 'PERS_CULTURE', 'PERS_CONFLICT', 'PERS_TOTAL',
       'PERS_WHITE', 'FRAC_WHITE', 'PERS_BLACK', 'FRAC_BLACK', 'PERS_HISP',
       'FRAC_HISP', 'PERS_AMIND',

In [8]:
# drop repetitive information captured in other columns
vic_df_lemas=vic_df_lemas.drop(['Unnamed: 0','wapo_id','city_mpv','state_code','police_department'],axis=1)

In [19]:
# drop rows where victim's race is nan
vic_df_lemas=vic_df_lemas.drop(np.where(vic_df_lemas['race'].isnull())[0],axis=0)
vic_df_lemas

,id,name,date,armed_wapo,age,gender,race,city_wapo,state,signs_of_mental_illness,...,PERS_AMIND,FRAC_AMIND,PERS_ASIAN,FRAC_ASIAN,PERS_HAWPI,FRAC_HAWPI,PERS_MULTI,FRAC_MULTI,PERS_UNK,FRAC_UNK
0,3,Tim Elliot,2015-01-02,gun,53.0,M,A,Shelton,WA,True,...,3,0.055556,0,0.000000,0,0.000000,0,0.0,0,0.000000
1,4,Lewis Lee Lembke,2015-01-02,gun,47.0,M,W,Aloha,OR,False,...,3,0.007463,16,0.039801,0,0.000000,0,0.0,3,0.007463
2,5,John Paul Quintero,2015-01-03,unarmed,23.0,M,H,Wichita,KS,False,...,2,0.002950,21,0.030973,0,0.000000,0,0.0,0,0.000000
3,8,Matthew Hoffman,2015-01-04,toy weapon,32.0,M,W,San Francisco,CA,True,...,7,0.002971,385,0.163413,140,0.059423,0,0.0,38,0.016129
4,9,Michael Rodriguez,2015-01-04,nail gun,39.0,M,H,Evans,CO,False,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647,6254,Brandon Evans,2020-10-24,knife,33.0,M,W,Lawrenceburg,IN,False,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000
5648,6255,Jakerion Shmond Jackson,2020-10-24,gun,19.0,M,B,Sylvester,GA,False,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000
5650,6266,Walter Wallace,2020-10-26,knife,27.0,M,B,Philadelphia,PA,True,...,6,0.000995,98,0.016249,11,0.001824,0,0.0,0,0.000000
5652,6268,Isaac Lemoine Christensen,2020-10-27,gun,38.0,M,W,Herriman,UT,False,...,0,0.000000,1,0.008929,0,0.000000,0,0.0,0,0.000000


In [21]:
# some columns still have missing values (age and armed_wapo probably are the only ones mattering)
vic_df_lemas.columns[np.sum(vic_df_lemas.isnull())>0]

Index(['name', 'armed_wapo', 'age', 'flee', 'longitude', 'latitude',
       'street_address', 'zipcode', 'county'],
      dtype='object')